In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pickle

In [4]:
##Load data set
## Objective : Detect th salary
data=pd.read_csv("Churn_Modelling.csv")
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [5]:
## drop columns rownumber, cusrtomerid, surname bcoz we don't need them to predict salary
data=data.drop(['RowNumber','CustomerId','Surname'], axis=1)
data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [6]:
## convert Gender column to 0,1 using label encode
label_encoder_gender= LabelEncoder()
data['Gender']=label_encoder_gender.fit_transform(data['Gender'])
data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,1,39,5,0.00,2,1,0,96270.64,0
9996,516,France,1,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,0,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,1,42,3,75075.31,2,1,0,92888.52,1


In [7]:
## convert \geography column to digits using onehot encoder
from sklearn.preprocessing import OneHotEncoder
onehot_encode_geo=OneHotEncoder()
geo_encoder=onehot_encode_geo.fit_transform(data[['Geography']])
geo_encoder

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 10000 stored elements and shape (10000, 3)>

In [8]:
## print the new columns of geography that got created after one hot encoding
onehot_encode_geo.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [9]:
geo_encoded_df=pd.DataFrame(geo_encoder.toarray(), columns=onehot_encode_geo.get_feature_names_out(['Geography']))
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [10]:
##Combine one hot encoded columns with the original data
data=pd.concat([data.drop('Geography',axis=1), geo_encoded_df], axis=1)
data


,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,1,39,5,0.00,2,1,0,96270.64,0,1.0,0.0,0.0
9996,516,1,35,10,57369.61,1,1,1,101699.77,0,1.0,0.0,0.0
9997,709,0,36,7,0.00,1,0,1,42085.58,1,1.0,0.0,0.0
9998,772,1,42,3,75075.31,2,1,0,92888.52,1,0.0,1.0,0.0


In [11]:
##save the encoders in individual pickle files to use later
with open('salary_label_encoder_gender.pkl', 'wb') as file:
    pickle.dump(label_encoder_gender, file)

with open('salary_onehot_encode_geo.pkl', 'wb') as file:
    pickle.dump(onehot_encode_geo, file)

In [12]:
##Divide the dataset into dependent and independent feature
X= data.drop('EstimatedSalary', axis=1)
Y=data['EstimatedSalary']

In [13]:
##Split the data into training and testing sets
X_test,X_train,Y_test,Y_train=train_test_split(X,Y, test_size=0.2, random_state=42)

##scale the features
scale=StandardScaler()
X_train=scale.fit_transform(X_train)
X_test=scale.fit_transform(X_test)

In [14]:
##save the scale into a pickle file
with open('salary_scale.pkl', 'wb') as file:
    pickle.dump(scale, file)

#### ANN REGRESSION PROBLEM STATEMENT

In [16]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime


In [17]:
##no of elements
X_train.shape[1]

12

In [19]:
## Build ANN model with hidden layer using Dense
## (X_train.shape[1],) means Single dimention with 12 inputs
model=Sequential([
    Dense(64, activation='relu',input_shape=(X_train.shape[1],)), ## HL1 which connects to input layer
    Dense(32, activation='relu'),##HL2 it doesn't require input as it's squential it'll take from previous layer
    Dense(1) ##output layer ## if no 2nd param mentioned means it takes linear regression function by defau;t
]
)
model.compile(optimizer='adam',loss='mean_absolute_error',metrics=['mae'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 64)                832       
                                                                 
 dense_4 (Dense)             (None, 32)                2080      
                                                                 
 dense_5 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [21]:
##Now to train a model we need to create logs ans setup as tensor dashboard
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
 ## Earlystopping is required to break the loop when loss valus not decreasing for 5 epochs at least
log_dir="regressionlogs/fit/"+datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback=TensorBoard(log_dir=log_dir,histogram_freq=1)

In [22]:
##set up early stopping
early_stopping_callback=EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)

In [23]:
## Train the model
history=model.fit(
    X_train,Y_train,
    validation_data=(X_test,Y_test), 
    epochs=100,
    callbacks=[tensorflow_callback,early_stopping_callback]
)

Epoch 1/100


63/63 [==============================] - 3s 17ms/step - loss: 98723.1016 - mae: 98723.1016 - val_loss: 100423.8594 - val_mae: 100423.8594
Epoch 2/100
63/63 [==============================] - 1s 10ms/step - loss: 98708.4062 - mae: 98708.4062 - val_loss: 100397.6719 - val_mae: 100397.6719
Epoch 3/100
63/63 [==============================] - 1s 10ms/step - loss: 98662.8516 - mae: 98662.8516 - val_loss: 100327.0938 - val_mae: 100327.0938
Epoch 4/100
63/63 [==============================] - 1s 9ms/step - loss: 98559.7188 - mae: 98559.7188 - val_loss: 100184.9922 - val_mae: 100184.9922
Epoch 5/100
63/63 [==============================] - 1s 9ms/step - loss: 98371.7812 - mae: 98371.7812 - val_loss: 99944.1172 - val_mae: 99944.1172
Epoch 6/100
63/63 [==============================] - 1s 8ms/step - loss: 98072.6953 - mae: 98072.6953 - val_loss: 99581.3984 - val_mae: 99581.3984
Epoch 7/100
63/63 [==============================] - 1s 10ms/step - loss: 97642.7344 - mae: 97642.7344 - 

In [24]:
## Load Tensorboard extension session
%load_ext tensorboard

In [26]:
## load magic function tensorboard
%tensorboard --logdir regressionlogs/fit/

Reusing TensorBoard on port 6006 (pid 8280), started 0:00:40 ago. (Use '!kill 8280' to kill it.)

In [ ]:
##Evaluate model on test data
test_loss, test_mae=model.evaluate(X_train,Y_test)
print(f'Test MAE:{test_mae}')